# Outlier or Catilin Clark? A Data Science Project
## Part 2 - Data Cleaning and Preprocessing

This notebook contains the code for the second part of this data science project - data cleaning and preprocessing. Section headings have been included for convenience and the full writeup is available [on my website](https://www.pineconedata.com/2024-05-02-basketball-data-cleaning-preprocessing/).

In summary, there will be a notebook (and post) for each part of the process - from initial steps like data acquisition, preprocessing, and cleaning to more advanced steps like feature engineering, machine learning, and creating visualizations. The dataset used in this project contains individual basketball player statistics (such as total points scored and blocks made) for the 2023-2024 NCAA women’s basketball season.

# Getting Started
Full requirements and environment setup information is detailed in the [full blog post](https://www.pineconedata.com/2024-04-11-basketball-data-acquisition/).

## Import Packages

In [2]:
import pandas as pd
import requests
import json
import os
import numpy as np
import openpyxl 

## Import Data

In [3]:
player_data = pd.read_excel('player_data_raw.xlsx')
player_data.head()

,PLAYER_NAME,Team,Class,Height,Position,PLAYER_ID,TEAM_NAME,GAMES,MINUTES_PLAYED,FIELD_GOALS_MADE,...,FREE_THROW_PERCENTAGE,OFFENSIVE_REBOUNDS,DEFENSIVE_REBOUNDS,TOTAL_REBOUNDS,ASSISTS,TURNOVERS,STEALS,BLOCKS,FOULS,POINTS
0,Kiara Jackson,UNLV (Mountain West),Jr.,5-7,G,ncaaw.p.67149,UNLV,29,895,128,...,75.0,27,102,129,135,42,31,5,47,323
1,Raven Johnson,South Carolina (SEC),So.,5-8,G,ncaaw.p.67515,South Carolina,30,823,98,...,64.3,33,128,161,148,53,60,5,34,243
2,Gina Marxen,Montana (Big Sky),Sr.,5-8,G,ncaaw.p.57909,Montana,29,778,88,...,72.4,6,54,60,111,38,16,2,26,297
3,McKenna Hofschild,Colorado St. (Mountain West),Sr.,5-2,G,ncaaw.p.60402,Colorado St.,29,1046,231,...,83.5,6,109,115,211,71,36,4,34,654
4,Kaylah Ivey,Boston College (ACC),Jr.,5-8,G,ncaaw.p.64531,Boston Coll.,33,995,47,...,60.7,12,45,57,186,64,36,1,48,143


# Data Cleaning

## Handle Missing Values

### Identify Missing Values

In [4]:
player_data[player_data.isna().any(axis=1)]

,PLAYER_NAME,Team,Class,Height,Position,PLAYER_ID,TEAM_NAME,GAMES,MINUTES_PLAYED,FIELD_GOALS_MADE,...,FREE_THROW_PERCENTAGE,OFFENSIVE_REBOUNDS,DEFENSIVE_REBOUNDS,TOTAL_REBOUNDS,ASSISTS,TURNOVERS,STEALS,BLOCKS,FOULS,POINTS
148,Ally Becki,Ball St. (MAC),Jr.,5-8,NaN,ncaaw.p.66590,Ball St.,31,972,143,...,75.4,21,122,143,148,108,60,11,69,398
245,Caitlin Weimar,Boston U. (Patriot),Sr.,6-4,F,ncaaw.p.64464,N.C. State,28,987,199,...,68.4,76,219,295,54,69,37,80,74,519
250,Abby Muse,Boise St. (Mountain West),Sr.,6-2,F,ncaaw.p.64516,Boise St.,31,785,90,...,54.3,74,191,265,34,66,36,87,72,230
254,Emily Bowman,Samford (SoCon),So.,6-5,C,ncaaw.p.64719,Samford,30,703,89,...,53.1,83,172,255,17,55,5,74,88,238
257,Christina Dalce,Villanova (Big East),Jr.,6-2,F,ncaaw.p.67708,Villanova,30,833,108,...,54.9,145,146,291,30,52,25,70,89,255
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
726,Kseniia Kozlova,James Madison (Sun Belt),Sr.,6-3,C,ncaaw.p.64131,James Madison,24,526,118,...,57.1,74,97,171,19,51,8,8,55,284
727,Simone Cunningham,Old Dominion (Sun Belt),Jr.,5-10,G,ncaaw.p.113163,Old Dominion,30,591,84,...,56.5,87,123,210,12,60,20,21,93,207
729,Otaifo Esenabhalu,Longwood (Big South),Fr.,6-2,F,ncaaw.p.113170,Longwood,30,547,60,...,60.0,79,128,207,10,59,23,13,78,147
755,Sedayjha Payne,Morgan St. (MEAC),Sr.,5-8,G,ncaaw.p.113276,Morgan St.,29,634,80,...,52.0,66,73,139,24,53,67,7,58,186


In [5]:
player_data.columns[player_data.isna().any()].tolist()

['Position', 'THREE_POINT_PERCENTAGE']

In [6]:
player_data[player_data.isna().any(axis=1)][['PLAYER_NAME', 'Team', 'Position', 'THREE_POINT_PERCENTAGE']]

,PLAYER_NAME,Team,Position,THREE_POINT_PERCENTAGE
148,Ally Becki,Ball St. (MAC),NaN,34.8
245,Caitlin Weimar,Boston U. (Patriot),F,NaN
250,Abby Muse,Boise St. (Mountain West),F,NaN
254,Emily Bowman,Samford (SoCon),C,NaN
257,Christina Dalce,Villanova (Big East),F,NaN
...,...,...,...,...
726,Kseniia Kozlova,James Madison (Sun Belt),C,NaN
727,Simone Cunningham,Old Dominion (Sun Belt),G,NaN
729,Otaifo Esenabhalu,Longwood (Big South),F,NaN
755,Sedayjha Payne,Morgan St. (MEAC),G,NaN


### Handle Missing Three-Point Percentages

In [7]:
player_data[player_data['THREE_POINT_PERCENTAGE'].isna()][['PLAYER_NAME', 'Team', 'Position', 'THREE_POINTS_MADE', 'THREE_POINT_ATTEMPTS', 'THREE_POINT_PERCENTAGE']]

,PLAYER_NAME,Team,Position,THREE_POINTS_MADE,THREE_POINT_ATTEMPTS,THREE_POINT_PERCENTAGE
245,Caitlin Weimar,Boston U. (Patriot),F,0,0,NaN
250,Abby Muse,Boise St. (Mountain West),F,0,0,NaN
254,Emily Bowman,Samford (SoCon),C,0,0,NaN
257,Christina Dalce,Villanova (Big East),F,0,0,NaN
258,Emily Saunders,Youngstown St. (Horizon),F,0,0,NaN
264,Lauren Betts,UCLA (Pac-12),C,0,0,NaN
265,Tenin Magassa,Rhode Island (Atlantic 10),C,0,0,NaN
275,Brooklyn Meyer,South Dakota St. (Summit League),F,0,0,NaN
282,Breya Cunningham,Arizona (Pac-12),F,0,0,NaN
290,Rochelle Norris,Central Mich. (MAC),C,0,0,NaN


### Handle Missing Positions

In [8]:
player_data[player_data['Position'].isna()][['PLAYER_NAME', 'Team', 'Position']]

,PLAYER_NAME,Team,Position
148,Ally Becki,Ball St. (MAC),NaN
359,Marie Kiefer,Ball St. (MAC),NaN
709,Ava Uhrich,Southern Utah (WAC),NaN
843,Madelyn Bischoff,Ball St. (MAC),NaN


In [9]:
player_data.loc[player_data['PLAYER_NAME'] == 'Ally Becki', 'Position'] = 'G'
player_data.loc[player_data['PLAYER_NAME'] == 'Marie Kiefer', 'Position'] = 'F'
player_data.loc[player_data['PLAYER_NAME'] == 'Ava Uhrich', 'Position'] = 'F'
player_data.loc[player_data['PLAYER_NAME'] == 'Madelyn Bischoff', 'Position'] = 'G'

In [10]:
player_data[player_data['Position'].isna()][['PLAYER_NAME', 'Team', 'Position']]

,PLAYER_NAME,Team,Position


## Handle Incorrect Values

### Handle Incorrect Heights

In [11]:
player_data['Height'].unique()

array(['5-7', '5-8', '5-2', '5-9', '5-6', '6-0', '5-10', '5-3', '5-11',
       '5-5', '5-4', '6-2', '6-1', '6-3', '6-4', '6-6', '6-5', '6-7',
       '6-8', '0-0'], dtype=object)

In [12]:
player_data[player_data['Height'].eq('0-0')]

,PLAYER_NAME,Team,Class,Height,Position,PLAYER_ID,TEAM_NAME,GAMES,MINUTES_PLAYED,FIELD_GOALS_MADE,...,FREE_THROW_PERCENTAGE,OFFENSIVE_REBOUNDS,DEFENSIVE_REBOUNDS,TOTAL_REBOUNDS,ASSISTS,TURNOVERS,STEALS,BLOCKS,FOULS,POINTS
709,Ava Uhrich,Southern Utah (WAC),Fr.,0-0,F,ncaaw.p.115529,Southern Utah,29,877,151,...,68.4,65,149,214,47,55,22,19,76,383
823,Payton Hull,Abilene Christian (WAC),Fr.,0-0,G,ncaaw.p.112709,Abilene Christian,29,837,155,...,75.3,33,67,100,59,87,47,8,71,444


In [13]:
player_data.loc[player_data['PLAYER_NAME'] == 'Ava Uhrich', 'Height'] = '6-0'
player_data.loc[player_data['PLAYER_NAME'] == 'Payton Hull', 'Height'] = '5-11'

In [14]:
player_data[player_data.eq('0-0').any(axis=1)]

,PLAYER_NAME,Team,Class,Height,Position,PLAYER_ID,TEAM_NAME,GAMES,MINUTES_PLAYED,FIELD_GOALS_MADE,...,FREE_THROW_PERCENTAGE,OFFENSIVE_REBOUNDS,DEFENSIVE_REBOUNDS,TOTAL_REBOUNDS,ASSISTS,TURNOVERS,STEALS,BLOCKS,FOULS,POINTS


### Handle Incorrect Classes

In [15]:
player_data['Class'].unique()

array(['Jr.', 'So.', 'Sr.', 'Fr.', '---'], dtype=object)

In [16]:
player_data[player_data['Class'] == '---']

,PLAYER_NAME,Team,Class,Height,Position,PLAYER_ID,TEAM_NAME,GAMES,MINUTES_PLAYED,FIELD_GOALS_MADE,...,FREE_THROW_PERCENTAGE,OFFENSIVE_REBOUNDS,DEFENSIVE_REBOUNDS,TOTAL_REBOUNDS,ASSISTS,TURNOVERS,STEALS,BLOCKS,FOULS,POINTS
236,Ayanna Khalfani,UNC Greensboro (SoCon),---,5-11,G,ncaaw.p.61460,N.C. Greensboro,31,911,139,...,60.0,87,126,213,109,96,41,30,75,344


In [17]:
player_data.loc[player_data['PLAYER_NAME'] == 'Ayanna Khalfani', 'Class'] = 'Sr.'

In [18]:
player_data['Class'].unique()

array(['Jr.', 'So.', 'Sr.', 'Fr.'], dtype=object)

# Data Preprocessing

## Data Type Conversion

In [19]:
player_data.dtypes

PLAYER_NAME                object
Team                       object
Class                      object
Height                     object
Position                   object
PLAYER_ID                  object
TEAM_NAME                  object
GAMES                       int64
MINUTES_PLAYED              int64
FIELD_GOALS_MADE            int64
FIELD_GOAL_ATTEMPTS         int64
FIELD_GOAL_PERCENTAGE     float64
THREE_POINTS_MADE           int64
THREE_POINT_ATTEMPTS        int64
THREE_POINT_PERCENTAGE    float64
FREE_THROWS_MADE            int64
FREE_THROW_ATTEMPTS         int64
FREE_THROW_PERCENTAGE     float64
OFFENSIVE_REBOUNDS          int64
DEFENSIVE_REBOUNDS          int64
TOTAL_REBOUNDS              int64
ASSISTS                     int64
TURNOVERS                   int64
STEALS                      int64
BLOCKS                      int64
FOULS                       int64
POINTS                      int64
dtype: object

### Convert Numeric-only Columns

In [21]:
numeric_columns = ['GAMES', 'MINUTES_PLAYED', 'FIELD_GOALS_MADE', 'FIELD_GOAL_ATTEMPTS', 
                  'FIELD_GOAL_PERCENTAGE', 'THREE_POINTS_MADE', 'THREE_POINT_ATTEMPTS', 
                  'THREE_POINT_PERCENTAGE', 'FREE_THROWS_MADE', 'FREE_THROW_ATTEMPTS', 
                  'FREE_THROW_PERCENTAGE', 'OFFENSIVE_REBOUNDS', 'DEFENSIVE_REBOUNDS', 
                  'TOTAL_REBOUNDS', 'ASSISTS', 'TURNOVERS', 'STEALS', 'BLOCKS', 'FOULS', 'POINTS']

In [22]:
player_data[numeric_columns] = player_data[numeric_columns].apply(pd.to_numeric)
player_data.dtypes

PLAYER_NAME                object
Team                       object
Class                      object
Height                     object
Position                   object
PLAYER_ID                  object
TEAM_NAME                  object
GAMES                       int64
MINUTES_PLAYED              int64
FIELD_GOALS_MADE            int64
FIELD_GOAL_ATTEMPTS         int64
FIELD_GOAL_PERCENTAGE     float64
THREE_POINTS_MADE           int64
THREE_POINT_ATTEMPTS        int64
THREE_POINT_PERCENTAGE    float64
FREE_THROWS_MADE            int64
FREE_THROW_ATTEMPTS         int64
FREE_THROW_PERCENTAGE     float64
OFFENSIVE_REBOUNDS          int64
DEFENSIVE_REBOUNDS          int64
TOTAL_REBOUNDS              int64
ASSISTS                     int64
TURNOVERS                   int64
STEALS                      int64
BLOCKS                      int64
FOULS                       int64
POINTS                      int64
dtype: object

In [23]:
player_data[player_data['THREE_POINT_PERCENTAGE'].isna()][['PLAYER_NAME', 'Team', 'Position', 'THREE_POINTS_MADE', 'THREE_POINT_ATTEMPTS', 'THREE_POINT_PERCENTAGE']].head()

,PLAYER_NAME,Team,Position,THREE_POINTS_MADE,THREE_POINT_ATTEMPTS,THREE_POINT_PERCENTAGE
245,Caitlin Weimar,Boston U. (Patriot),F,0,0,NaN
250,Abby Muse,Boise St. (Mountain West),F,0,0,NaN
254,Emily Bowman,Samford (SoCon),C,0,0,NaN
257,Christina Dalce,Villanova (Big East),F,0,0,NaN
258,Emily Saunders,Youngstown St. (Horizon),F,0,0,NaN


### Convert Text-only Columns

In [24]:
string_columns = ['PLAYER_NAME', 'Team', 'Class', 'Height', 'Position', 'PLAYER_ID', 'TEAM_NAME']

In [25]:
player_data[string_columns].sample(10)

,PLAYER_NAME,Team,Class,Height,Position,PLAYER_ID,TEAM_NAME
619,Taleah Washington,Wagner (NEC),Sr.,5-7,G,ncaaw.p.60482,Wagner
571,Kirsten Lewis-Williams,Buffalo (MAC),Fr.,5-10,G,ncaaw.p.112576,Buffalo
424,Khalis Cain,UNC Greensboro (SoCon),Sr.,6-3,F,ncaaw.p.64664,N.C. Greensboro
70,Nitzan Amar,Manhattan (MAAC),So.,5-10,G,ncaaw.p.67171,Manhattan
775,Trinity Hardy,Army West Point (Patriot),Jr.,5-8,G,ncaaw.p.68094,Army
798,Gabi Fields,Northern Colo. (Big Sky),So.,5-5,G,ncaaw.p.71666,Northern Colorado
552,Jillian Hayes,Cincinnati (Big 12),Sr.,6-1,F,ncaaw.p.64376,Cincinnati
520,Honesty Scott-Grayson,Auburn (SEC),Sr.,5-9,G,ncaaw.p.56767,Auburn
803,Teya Sidberry,Boston College (ACC),So.,6-1,F,ncaaw.p.71438,Boston Coll.
794,Emily Carver,App State (Sun Belt),Jr.,5-10,G,ncaaw.p.66617,Appalachian St.


In [26]:
player_data[string_columns] = player_data[string_columns].astype('string')
player_data[string_columns].dtypes

PLAYER_NAME    string[python]
Team           string[python]
Class          string[python]
Height         string[python]
Position       string[python]
PLAYER_ID      string[python]
TEAM_NAME      string[python]
dtype: object

## Value Substitution

### Substitute Abbreviated Position Names

In [27]:
player_data['Position'].unique()

<StringArray>
['G', 'F', 'C']
Length: 3, dtype: string

In [28]:
position_names = {
    'F': 'Forward',
    'G': 'Guard',
    'C': 'Center'
}

In [29]:
player_data['Position'] = player_data['Position'].replace(position_names)
player_data['Position'].unique()

<StringArray>
['Guard', 'Forward', 'Center']
Length: 3, dtype: string

### Substitute Abbreviated Class Names

In [30]:
player_data['Class'].unique()

<StringArray>
['Jr.', 'So.', 'Sr.', 'Fr.']
Length: 4, dtype: string

In [31]:
class_names = {
    'Fr.': 'Freshman',
    'So.': 'Sophomore',
    'Jr.': 'Junior',
    'Sr.': 'Senior'
}

In [32]:
player_data['Class'] = player_data['Class'].replace(class_names)
player_data['Class'].unique()

<StringArray>
['Junior', 'Sophomore', 'Senior', 'Freshman']
Length: 4, dtype: string

## Unit Conversion

In [33]:
def height_to_inches(height):
    feet, inches = map(int, height.split('-'))
    return feet * 12 + inches

In [34]:
player_data['Height'] = player_data['Height'].apply(height_to_inches)
player_data['Height'].unique()

array([67, 68, 62, 69, 66, 72, 70, 63, 71, 65, 64, 74, 73, 75, 76, 78, 77,
       79, 80])

# Wrap Up

In [35]:
player_data.to_excel('player_data_clean.xlsx', index=False)